In [492]:
import pandas as pd
const_info = pd.read_csv('cons.csv') 

In [493]:
const_email = pd.read_csv('cons_email.csv') 

In [494]:
# remove rows without a primary email
const_email = const_email.loc[const_email['is_primary'] == 1]

In [495]:
const_subs = pd.read_csv('cons_email_chapter_subscription.csv')

In [496]:
# remove irrelevant subs statuses
const_subs = const_subs.loc[const_subs['chapter_id'] == 1]

In [497]:
# cutting down dataframes to only needed columns
const_subs = const_subs[['cons_email_id','isunsub']]
const_info = const_info[['cons_id','source']]
const_email = const_email[['cons_id','cons_email_id', 'email', 'create_dt','modified_dt']]

In [498]:
len(const_subs)

275484

In [499]:
len(const_info)

700000

In [500]:
len(const_email)

605639

In [501]:
# right join because only want rows with primary email
const_info_email = const_info.merge(const_email,how='right', 
                            left_on=['cons_id'], right_on=['cons_id'])

In [502]:
# outer join because assumed subscribed even if there is no email
const_info_email_subs = const_info_email.merge(const_subs,how='outer', 
                            left_on=['cons_email_id'], right_on=['cons_email_id'])


In [503]:
len(const_info_email_subs)

707133

In [504]:
const_info_email_subs.isnull().sum()
# quick check of data looks ok - there are only 275484 valid values for isunsub. 707133 - 275484 = 431649

cons_id          101494
source           404193
cons_email_id         0
email            101494
create_dt        101494
modified_dt      101494
isunsub          431649
dtype: int64

In [505]:
# some final sanity checking of data
# email duplicates + 1 should equal NaNs
(const_info_email_subs['email'].duplicated().sum() + 1) == const_info_email_subs['email'].isnull().sum()

True

In [507]:
# ensure correct data types
const_info_email_subs['email'] = const_info_email_subs['email'].astype(str)
const_info_email_subs['source'] = const_info_email_subs['source'].astype(str)
const_info_email_subs[['create_dt', 'modified_dt']] = const_info_email_subs[['create_dt', 'modified_dt']].apply(pd.to_datetime)
const_info_email_subs.head()

,cons_id,source,cons_email_id,email,create_dt,modified_dt,isunsub
0,1.0,google,546912,daniel72@hudson.com,1982-11-26 14:01:20,2008-05-31 02:30:35,NaN
1,2.0,facebook,415587,caustin@spears-carson.com,1987-01-23 13:51:26,1982-12-17 18:55:42,1.0
2,3.0,nan,81594,klewis@ford.biz,2013-05-02 09:20:11,1992-03-24 07:15:14,1.0
3,5.0,nan,182741,stephenhamilton@gmail.com,1976-05-23 12:23:38,1990-04-01 16:41:29,1.0
4,6.0,google,246591,henryherring@gmail.com,1989-10-07 13:12:01,1992-06-24 21:50:42,NaN


In [508]:
# subset final wanted columns and modify names to desired format
people = const_info_email_subs[['email','source','isunsub','create_dt', 'modified_dt']]
people = people.rename(columns={'source': 'code', 'isunsub': 'is_unsub','create_dt':'created_dt','modified_dt':'updated_dt'})

In [509]:
people = people.sort_values(by=['email'])

In [510]:
# save people as csv 
people.to_csv('people.csv',index=False)

In [511]:
# create “acquisition_facts” file with date of acquisition and number of acquisitions on the date
acquisition_facts = people[['created_dt']]
acquisition_facts = acquisition_facts.rename(columns={'created_dt': 'acquisition_date'})

In [512]:
# remove time from datetime
acquisition_facts['acquisition_date'] = pd.to_datetime(acquisition_facts['acquisition_date']).dt.date

In [513]:
# use group by and size to find number of acquisitions per date
acquisition_facts = acquisition_facts.groupby(['acquisition_date']).size().reset_index(name='acquisitions')

,acquisition_date,acquisitions
0,1970-01-01,45
1,1970-01-02,29
2,1970-01-03,26
3,1970-01-04,31
4,1970-01-05,31
...,...,...
18440,2020-06-27,39
18441,2020-06-28,27
18442,2020-06-29,40
18443,2020-06-30,31


In [514]:
# sanity check of results
acquisition_facts.acquisitions.sum() == (len(people) - people.updated_dt.isnull().sum())

True

In [515]:
# save as csv 
acquisition_facts.to_csv('acquisition_facts.csv',index=False)